###### References: 
- Fluent Python, 2nd Edition, by Luciano Ramalho.

# Chapter 1: The Python Data Model

## Collection API
<img src="abc.png" width="75%">

### `collection.abc`
Interfaces:
  * `iterable` to support for, unpacking, andd  other forms  of iteration
  * `sized` to support the len built-in function
  * `container` to support the  in operator

specialations:
  * `Sequence`, formalizing the interface of built-ins like `list` and `str`
  * `Mapping`, implemented by `dict`, `collections.defaultdict`, etc
  * `Set`, the interface of the `set` and `frozenset` built-in types

# Chapter 2: An Array of Sequences

## Pattern Matching with Sequence

\> Python 3.10 : `match/case` statement  
PEP 634 -- Structural Pattern Matching: Specification

In [1]:
def handle_command(self, message):
    match message:  # expression after the match keywordd is the subject
        case ['BEEPER', frequency, times]: # pattern matches any subject that is a sequence with three items
            self.beep(times, frequency)
        case ['NECK', angle]: 
            self.rotate_neck(angle)
        case ['LED', ident, intensity]:
            self.leds[ident].set_brightness(ident, intensity)
        case ['LED', ident, red, green, blue]:
            self.leds[ident].set_color(ident, red, green, blue)
        case _:
            raise InvalidCommand(message)

Similar with C language `switch/case`, but with destructuring.

In [2]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        match record:
            case [name, _, _, (lat, lon)] if lon <= 0:
                print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')
main()

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


# Chapter 3: Dictionaries and Sets

## Modern `dict` Syntax

### Merging Mappings with `|`

Python 3.9 supports using of `|` and `|=` to merge mapping

In [3]:
d1 = {'a':1, 'b': 3}
d2 = {'a':2, 'b': 4, 'c': 6}
d1 | d2

{'a': 2, 'b': 4, 'c': 6}

In [4]:
d1 |= d2
d1

{'a': 2, 'b': 4, 'c': 6}

##  Pattern Matching  with  Mappings

In [5]:
def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:  # <1>
            return names
        case {'type': 'book', 'api': 1, 'author': name}:  # <2>
            return [name]
        case {'type': 'book'}:  # <3>
            raise ValueError(f"Invalid 'book' record: {record!r}")
        case {'type': 'movie', 'director': name}:  # <4>
            return [name]
        case _:  # <5>
            raise ValueError(f'Invalid record: {record!r}')

In [6]:
b1 = dict(api=1, author='Douglas Hofstadter',
         type='book', title='Gödel, Escher, Bach')
get_creators(b1)

['Douglas Hofstadter']

In [7]:
from collections import OrderedDict

In [8]:
b2 = OrderedDict(api=2, type='book',
         title='Python in a Nutshell',
         authors='Martelli Ravenscroft Holden'.split())
get_creators(b2)

['Martelli', 'Ravenscroft', 'Holden']

In [9]:
get_creators({'type': 'book', 'pages': 770})

ValueError: Invalid 'book' record: {'type': 'book', 'pages': 770}

# Chapter 5: Data Class Builders
> TBC

# Chapter 7: Functions as First-Class Objects
## Positional-Only Parameters
\> Python 3.8

In [10]:
def divmod(a, b, /):
    return (a // b, a % b)

In [11]:
def tag(name, /, *content, class_=None, **attrs):
    pass

# Chapter 8: Type Hints in Functions
> TBC

# Chapter 9: Decorators and Closures
## Decorators in the Standard Library
### Memoization with  functools.cache

In [12]:
import functools

from clockdeco import clock

In [13]:
@functools.cache  # > Python 3.9
@clock  # stack decorators
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [14]:
fibonacci(6)

[0.00000127s] fibonacci(0) -> 0
[0.00000140s] fibonacci(1) -> 1
[0.00051791s] fibonacci(2) -> 1
[0.00000227s] fibonacci(3) -> 2
[0.00056855s] fibonacci(4) -> 3
[0.00000125s] fibonacci(5) -> 5
[0.00060654s] fibonacci(6) -> 8


8

In [15]:
fibonacci(7)

[0.00000291s] fibonacci(7) -> 13


13

# Chapter 13: Interfaces, Protocols, and ABCs

>TBC
 * Typing Map
 * Two Kinds of Protocols
 * Static Protocols
 
# Chapter 14: Inheritance: For  Better or for Worse
> TBC
* Multiple Inheritance and Method Resolution Order
* Mixin Classes
* Multiple Inheritance in the Real World

# Chapter 15: More About  Type Hints
> TBC

# Chapter 16: Operator Overloading

In [16]:
from array import array
import reprlib
import math
import functools
import operator
import itertools
from collections import abc

In [17]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar):
        try:
            factor = float(scalar)
        except TypeError: # if scalar cannot be converted to float
            return NotImplemented
        return Vector(n * factor for n in self) # try __rmul__

    def __rmul__(self, scalar):
        return self * scalar   # delegating to __mul__

    def __matmul__(self, other):
        if (isinstance(other, abc.Sized) and
            isinstance(other, abc.Iterable)):
            if len(self) == len(other):
                return sum(a * b for a, b in zip(self, other))
            else:
                raise ValueError('@ requires vectors of equal length.')
        else:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other

In [18]:
v1 = Vector([1, 2, 3])
14 * v1

Vector([14.0, 28.0, 42.0])

In [19]:
v1 * 10

Vector([10.0, 20.0, 30.0])

In [20]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [21]:
from fractions import Fraction

In [22]:
v1 *  Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

## Using `@` as an Infix Operator
dot product or matrix multiplication.

In [23]:
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz == 38.0  # 1*5 + 2*6 + 3*7

True

In [24]:
[10, 20, 30] @ vz

380.0

In [25]:
va @ 3

TypeError: unsupported operand type(s) for @: 'Vector' and 'int'

# Chapter 18:  `with`, `match` and `else` Blocks
> TBC
* Pattern Matching in lis.py: a Cases Study
*  The contextlib  Utilities

# Chapter 19: Concurrency Models in Python
> TBC

# Chapter 20: Concurrency Executors
> TBC